# Summary
### Data product:<br> setting up a Tornado warning system with AWS Simple Notification System (SNS) using Python Boto 3.


<p>

This project uses the Simple Notification Service (SNS) of AWS. Several basic functions like creating topics, subscriptions, publishing are explained. These functions are used to set up a Tornado warning system, which sends warning messages about incoming tornados to email endpoints. There are two inputs into the SNS: a contact list of subscribers and a tornado information table. Depending on the speed of the Tornado, different messages are sent out to the endpoints . It is possible to develop the functionality of the warning system further.
</p>
    
<p>
The project relies heavily on the Boto3 documentation.
According to the doc:

“You use the AWS SDK for Python (Boto3) to create, configure, and manage AWS services, such as Amazon Elastic Compute Cloud (Amazon EC2) and Amazon Simple Storage Service (Amazon S3). The SDK provides an object-oriented API as well as low-level access to AWS services.”

This project creates an
</p>


<p> 
The project relies heavily on the  
<a href = https://boto3.amazonaws.com/v1/documentation/api/latest/index.html target=_blank> 
Boto3 documentation.</a> <br> 
According to the doc: 
</p> 

<p> 
“You use the AWS SDK for Python (Boto3) to create, configure, and manage AWS services, such as Amazon Elastic Compute Cloud (Amazon EC2) and Amazon Simple Storage Service (Amazon S3). The SDK provides an object-oriented API as well as low-level access to AWS services.” 
</p> 

<p>This project creates an  

### S3_helpers_pckg 

<p> 
The functionality is extended with a newly created helper package by me. This package is a work in progress. <br>
The package stores a class with useful helper functions, mostly manipulating the dicts of responses.<br> 
The functions are mostly self defined, but other functions for example from Github and the doc are integrated.<br> 
In this case credits are given.<br> 
The pckg is a work in progress. 
</p> 


# Import packages

In [1]:
# Import packages

import pandas as pd
import pandasql
from pandasql import sqldf

import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

import numpy as np
import random
import os as os
import time
import pprint
import sys
import re
import json
import glob 
import jinja2 # for styling data frames 
from IPython.display import display, HTML # Displaying HTML in Jupyter notebook

from pathlib import *

# security
from dotenv import load_dotenv
import logging

# display html in code cells
from IPython.display import HTML, display, Markdown, Latex, Image

#### Import Boto

In [2]:
# import boto3
import boto3
import botocore

### S3_helpers_pckg

<p>
stores a class with useful helper functions, mostly manipulating the dicts of responses.<br>
The functions are mostly self defined, but other functions for example from Github and the doc are integrated.<br>
In this case credits are given.<br>
The pckg is a work in progress.
</p>

In [3]:
import S3_helpers_pckg

In [4]:
initpy_path = S3_helpers_pckg

print(type(initpy_path))
print(str(initpy_path)[1:30])


<class 'module'>
module 'S3_helpers_pckg' from


In [5]:
from S3_helpers_pckg import S3_helpers

### Checkout the directories

In [6]:
cwd = os.getcwd()
# print(cwd)

In [7]:
# os.chdir('G:\Other computers\Mein Laptop (1)\data_camp_projects\Turing_DataAnalysis')
os.chdir('G:\Other computers\Mein Laptop (1)\data_camp_projects\AWS_boto3')

In [8]:
### List directories

directories_list = os.listdir()
directories_list

['AWS_boto3_S3_FileManagement.ipynb',
 'txt',
 '.ipynb_checkpoints',
 'data',
 '.env',
 'S3_helpers_pckg',
 'upload_files',
 'download_files',
 'csv',
 'html',
 '.git',
 'README.md',
 '.gitignore']

In [9]:
file_list = glob.glob("*")
print(file_list)

['AWS_boto3_S3_FileManagement.ipynb', 'txt', 'data', 'S3_helpers_pckg', 'upload_files', 'download_files', 'csv', 'html', 'README.md']


### Creating requirement files

In [10]:
print("Current version of Python is ", sys.version)
print(pd.__version__)
print(np.__version__)
print(sns.__version__)

Current version of Python is  3.10.4 | packaged by conda-forge | (main, Mar 30 2022, 08:38:02) [MSC v.1916 64 bit (AMD64)]
1.4.3
1.21.5
0.11.2


#### Making directories

In [11]:
if not os.path.exists("txt"):
    # if the demo_folder directory is not present 
    # then create it.
    os.makedirs("txt")
    
if not os.path.exists("upload_files"):
    os.makedirs("upload_files")
    
if not os.path.exists("data"):
    os.makedirs("data")


In [12]:
!conda list > txt/requirements_file_conda_boto3.txt
!pip list > txt/requirements_file_pip_boto3.txt

# Setting up AWS resources

## Import AWS keys


In [13]:
 %run S3_helpers_pckg/settings.py

# Setting up a tornado warning system with AWS Simple Notification Service (SNS)
<p>
SNS is a fully managed Pub/Sub service for A2A and A2P messaging.<br>
<a href="https://aws.amazon.com/sns/" target="_bank">AWS</a>
</p>

    




## Preparations: 
### Creating a client

<p>
"Clients provide a low-level interface to AWS whose methods map close to 1:1 with service APIs. All service operations are supported by clients. Clients are generated from a JSON service definition file." (Boto3 doc)
</p>

<p>
The name of the service and the keys are required.<br>
The service name here is 'sns'.<br>
</p>

In [15]:
s3_sns_client = boto3.client('sns', region_name='us-east-1',
                  aws_access_key_id=Access_Key_ID,
                  aws_secret_access_key=Secret_Access_Key)

In [16]:
s3_sns_client

#### Create topics

<p>
"An Amazon SNS topic is a logical access point that acts as a communication channel.<br>
A topic lets you group multiple endpoints (such as AWS Lambda, Amazon SQS, HTTP/S, or an email address)."
</p>

In [17]:
tornado_reponse = \
s3_sns_client.create_topic(Name='tornado_notification_service_system')


# tornado_response['TopicArn']
tornado_system_arn = tornado_reponse['TopicArn']
print(tornado_system_arn)

arn:aws:sns:us-east-1:047675893846:tornado_notification_service_system


#### List the active topics

<p>
Only the names.
</p>

In [18]:
topics_len_1 = len(s3_sns_client.list_topics()['Topics'])

for i in range(topics_len_1):
    print(s3_sns_client.list_topics()['Topics'][i]['TopicArn'].split(':')[-1])

Default_CloudWatch_Alarms_Topic
alarm_greater_than_one_dollar
tornado_notification_service
tornado_notification_service_system


### Building the tornado information table

In [19]:
speed = [50, 100, 200, 300, 400, 500] 

alarm = ['Low','Guarded', 'Elevated', 'High', 'Severe', 'Intense']

message = ['Danger is low! However, be careful!',
           'Be prepared. Look for falling trees!',
           'Increasing threat. Take security measures.',
           'Alarm. The risk to life is very high.',
           'Please, leave the area and seek shelter.',
           'Warning: natural disaster alarm!'
          ]

fujita = ['F0', 'F1', 'F2', 'F3', 'F4', 'F5']

category = ['weak', 'weak', 'strong', 'strong', 'violent', 'violent']


tornado_alarms_df = \
pd.DataFrame(zip(fujita, category, alarm, speed, message), 
             columns=['Fujita', 'Category', 'Alarm', 'Speed (km/h)', 'Message'],
             index=list(np.arange(0,6)))

tornado_alarms_df

,Fujita,Category,Alarm,Speed (km/h),Message
0,F0,weak,Low,50,"Danger is low! However, be careful!"
1,F1,weak,Guarded,100,Be prepared. Look for falling trees!
2,F2,strong,Elevated,200,Increasing threat. Take security measures.
3,F3,strong,High,300,Alarm. The risk to life is very high.
4,F4,violent,Severe,400,"Please, leave the area and seek shelter."
5,F5,violent,Intense,500,Warning: natural disaster alarm!


In [20]:
tornado_alarms_df.loc[1, 'Message']
tornado_alarms_df.loc[1, 'Alarm']

'Guarded'

### Building the contacts emailing list

In [21]:
import pandas as pd

emails = ['Gotermer3534@yahoo.com', 'iomer3423@mailfence.com', 'ermerdir.itoplasto@aol.com']

firstname = ['Uperto', 'Tessto', 'Aeedor']
lastname =['Gotermer', 'Iomer', 'Ermedir']
planet = ['Pluto', 'Saturn', 'Venus']
           
subscribers_df = \
pd.DataFrame(zip(firstname, lastname, planet, emails), 
             columns=['First name', 'Last name', 'Planet', 'Emails'])


subscribers_df

,First name,Last name,Planet,Emails
0,Uperto,Gotermer,Pluto,Gotermer3534@yahoo.com
1,Tessto,Iomer,Saturn,iomer3423@mailfence.com
2,Aeedor,Ermedir,Venus,ermerdir.itoplasto@aol.com


### Subscribing contacts to the topic

In [22]:
for row in subscribers_df.iterrows():
    email_endpoint = row[1]['Emails']
    # print(email_endpoint)tornado_system_arn

In [23]:
for row in subscribers_df.iterrows():


                responses_collected = []

                email_endpoint = row[1]['Emails']
            
                print(email_endpoint)

Gotermer3534@yahoo.com
iomer3423@mailfence.com
ermerdir.itoplasto@aol.com


In [24]:

def subscribe_contacts_to_topic_2(df, emails):

        for row in subscribers_df.iterrows():


                responses_collected = []

                email_endpoint = row[1][emails]
                print(email_endpoint)

                
subscribe_contacts_to_topic_2(df=subscribers_df, emails='Emails')

Gotermer3534@yahoo.com
iomer3423@mailfence.com
ermerdir.itoplasto@aol.com


In [25]:

def subscribe_contacts_to_topic(client, df, emails):

        for row in df.iterrows():
                    # print(row)

                    responses_collected = []

                    email_endpoint = row[1][emails]
                    # print(email_endpoint)

                    subscription_response = \
                    eval(client).subscribe(
                    TopicArn = tornado_system_arn,
                    Protocol = 'email',
                    Endpoint = email_endpoint )

                    responses_collected.append(subscription_response)

        return responses_collected

In [26]:
collected_responses = \
subscribe_contacts_to_topic(client="s3_sns_client", df=subscribers_df, emails='Emails')


In [27]:
collected_responses 

[{'SubscriptionArn': 'pending confirmation',
  'ResponseMetadata': {'RequestId': '00ac779f-7a02-591e-9734-2c63789d1f7f',
   'HTTPStatusCode': 200,
   'HTTPHeaders': {'x-amzn-requestid': '00ac779f-7a02-591e-9734-2c63789d1f7f',
    'content-type': 'text/xml',
    'content-length': '298',
    'date': 'Sat, 26 Nov 2022 12:02:38 GMT'},
   'RetryAttempts': 0}}]

### Publishing the alarms
<p>
applying a user-defined functions.
</p>

<p>
Input into the alarm function is the speed of the tornado.<br>
It is here generated with a random function.<br>
In future a real time data source is necessary.
</p>

In [28]:
# import random
# Input into the alarm function is
speed = round(random.uniform(0,600),2)
print(speed)


292.3


In [38]:
def alarm(speed):
    
        if (speed> 0) and (speed<= 50):
            print(1)
            alarm_level_0 = tornado_alarms_df.loc[0, 'Alarm']
            response_0 = \
            s3_sns_client.publish(TopicArn = tornado_system_arn,
                                  Message = tornado_alarms_df.loc[0, 'Message'],
                                  Subject = "Tornado alarm: speed {} km/h. Alarm level: {}".format(speed, alarm_level_0))
            
            return ('Response:', alarm_level_0, 'Full response:', response_0)
            
        elif (speed>= 51) and (speed<= 100):
                alarm_level_1 = tornado_alarms_df.loc[1, 'Alarm']
                response_1 = \
                s3_sns_client.publish(TopicArn = tornado_system_arn,
                                      Message = tornado_alarms_df.loc[1, 'Message'],
                                      Subject = "Tornado alarm: speed {} km/h. Alarm level: {}".format(speed, alarm_level_1))
                
                return ('Response:', alarm_level_1, 'Full response:', response_1)
                
        elif (speed>= 101) and (speed<= 200):
            alarm_level_2 = tornado_alarms_df.loc[2, 'Alarm']
            response_2 = \
            s3_sns_client.publish(TopicArn = tornado_system_arn,
                                  Message = tornado_alarms_df.loc[2, 'Message'],
                                  Subject = "Tornado alarm: speed {} km/h. Alarm level: {}".format(speed, alarm_level_2))
            
            return ('Response:', alarm_level_2, 'Full response:', response_2)
        
        elif (speed>= 201) and (speed<= 300):
            alarm_level_3 = tornado_alarms_df.loc[3, 'Alarm']
            response_3 = \
            s3_sns_client.publish(TopicArn = tornado_system_arn,
                                  Message = tornado_alarms_df.loc[3, 'Message'],
                                  Subject = "Tornado alarm: speed {} km/h. Alarm level: {}".format(speed, alarm_level_3))
            
            return ('Response:', alarm_level_3, 'Full response:', response_3)
    
        elif (speed>= 301) and (speed<= 400):
            alarm_level_4 = tornado_alarms_df.loc[4, 'Alarm']
            response_4 = \
            s3_sns_client.publish(TopicArn = tornado_system_arn,
                                  Message = tornado_alarms_df.loc[4, 'Message'],
                                  Subject = "Tornado alarm: speed {} km/h. Alarm level: {}".format(speed, alarm_level_4))
            
            return ('Response:', alarm_level_4, 'Full response:', response_4)
    
        elif speed> 401:
            alarm_level_5 = tornado_alarms_df.loc[5, 'Alarm']
            response_5 = \
            s3_sns_client.publish(TopicArn = tornado_system_arn,
                                  Message = tornado_alarms_df.loc[5, 'Message'],
                                  Subject = "Tornado alarm: speed {} km/h. Alarm level: {}".format(speed, alarm_level_5))
            return ({'Response:': alarm_level_5}, {'Full response:', response_5})


In [39]:
alarm_response_1 = alarm(speed=speed)

In [40]:
print(alarm_response_1)

('Response:', 'High', 'Full response:', {'MessageId': 'c41ecf28-7a31-5bf9-a554-94d479c0005a', 'ResponseMetadata': {'RequestId': '665bde06-9771-505f-bc16-85fa94809bfc', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '665bde06-9771-505f-bc16-85fa94809bfc', 'content-type': 'text/xml', 'content-length': '294', 'date': 'Sat, 26 Nov 2022 12:08:16 GMT'}, 'RetryAttempts': 0}})
